### Outline 

Graph AutoEncoders GAE &  
Variational Graph Autoencoders VGAE    

[paper](https://arxiv.org/pdf/1611.07308.pdf)  
[code](https://github.com/rusty1s/pytorch_geometric/blob/master/examples/autoencoder.py)

In [1]:
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges

# Graph AutoEncoder GAE

### Load the data

In [2]:
dataset = Planetoid("\..", "CiteSeer", transform=T.NormalizeFeatures())
dataset.data

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])

In [3]:
data = dataset[0]
data.train_mask = data.val_mask = data.test_mask = None
data

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327])

In [4]:
data = train_test_split_edges(data)

C:\Users\Travail\anaconda3\envs\pyg\lib\site-packages\torch_geometric\deprecation.py:13: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [5]:
data

Data(x=[3327, 3703], y=[3327], val_pos_edge_index=[2, 227], test_pos_edge_index=[2, 455], train_pos_edge_index=[2, 7740], train_neg_adj_mask=[3327, 3327], val_neg_edge_index=[2, 227], test_neg_edge_index=[2, 455])

### Define the Encoder

In [6]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv2 = GCNConv(2 * out_channels, out_channels, cached=True) # cached only for transductive learning

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

### Define the Autoencoder 

In [7]:
from torch_geometric.nn import GAE

In [8]:
# parameters
out_channels = 2
num_features = dataset.num_features
epochs = 100

# model
model = GAE(GCNEncoder(num_features, out_channels))

# move to GPU (if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

# inizialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [9]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    #if args.variational:
    #   loss = loss + (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [10]:
for epoch in range(1, epochs + 1):
    loss = train()

    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

Epoch: 001, AUC: 0.6390, AP: 0.6783
Epoch: 002, AUC: 0.6467, AP: 0.6879
Epoch: 003, AUC: 0.6395, AP: 0.6834
Epoch: 004, AUC: 0.6372, AP: 0.6832
Epoch: 005, AUC: 0.6356, AP: 0.6827
Epoch: 006, AUC: 0.6340, AP: 0.6823
Epoch: 007, AUC: 0.6329, AP: 0.6825
Epoch: 008, AUC: 0.6318, AP: 0.6827
Epoch: 009, AUC: 0.6315, AP: 0.6835
Epoch: 010, AUC: 0.6311, AP: 0.6845
Epoch: 011, AUC: 0.6308, AP: 0.6852
Epoch: 012, AUC: 0.6309, AP: 0.6865
Epoch: 013, AUC: 0.6312, AP: 0.6881
Epoch: 014, AUC: 0.6313, AP: 0.6898
Epoch: 015, AUC: 0.6310, AP: 0.6913
Epoch: 016, AUC: 0.6310, AP: 0.6929
Epoch: 017, AUC: 0.6307, AP: 0.6938
Epoch: 018, AUC: 0.6310, AP: 0.6954
Epoch: 019, AUC: 0.6305, AP: 0.6962
Epoch: 020, AUC: 0.6301, AP: 0.6969
Epoch: 021, AUC: 0.6304, AP: 0.6976
Epoch: 022, AUC: 0.6302, AP: 0.6979
Epoch: 023, AUC: 0.6301, AP: 0.6985
Epoch: 024, AUC: 0.6300, AP: 0.6989
Epoch: 025, AUC: 0.6300, AP: 0.6994
Epoch: 026, AUC: 0.6298, AP: 0.6996
Epoch: 027, AUC: 0.6302, AP: 0.7001
Epoch: 028, AUC: 0.6307, AP:

### Use Tensorboard

In [18]:
from torch.utils.tensorboard import SummaryWriter

In [22]:
# parameters
out_channels = 3
num_features = dataset.num_features
epochs = 100

# model
model = GAE(GCNEncoder(num_features, out_channels))

# move to GPU (if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

# inizialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [23]:
writer = SummaryWriter('runs/GAE1_experiment_'+'3d_100_epochs')

In [24]:
for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    
    
    writer.add_scalar('auc train',auc,epoch) # new line
    writer.add_scalar('ap train',ap,epoch)   # new line

Epoch: 001, AUC: 0.6514, AP: 0.6807
Epoch: 002, AUC: 0.6374, AP: 0.6756
Epoch: 003, AUC: 0.6315, AP: 0.6752
Epoch: 004, AUC: 0.6319, AP: 0.6782
Epoch: 005, AUC: 0.6332, AP: 0.6818
Epoch: 006, AUC: 0.6318, AP: 0.6820
Epoch: 007, AUC: 0.6300, AP: 0.6820
Epoch: 008, AUC: 0.6301, AP: 0.6831
Epoch: 009, AUC: 0.6307, AP: 0.6849
Epoch: 010, AUC: 0.6315, AP: 0.6863
Epoch: 011, AUC: 0.6321, AP: 0.6882
Epoch: 012, AUC: 0.6322, AP: 0.6897
Epoch: 013, AUC: 0.6323, AP: 0.6917
Epoch: 014, AUC: 0.6324, AP: 0.6937
Epoch: 015, AUC: 0.6330, AP: 0.6953
Epoch: 016, AUC: 0.6339, AP: 0.6972
Epoch: 017, AUC: 0.6346, AP: 0.6989
Epoch: 018, AUC: 0.6352, AP: 0.7005
Epoch: 019, AUC: 0.6355, AP: 0.7021
Epoch: 020, AUC: 0.6358, AP: 0.7028
Epoch: 021, AUC: 0.6358, AP: 0.7040
Epoch: 022, AUC: 0.6359, AP: 0.7045
Epoch: 023, AUC: 0.6363, AP: 0.7051
Epoch: 024, AUC: 0.6371, AP: 0.7061
Epoch: 025, AUC: 0.6394, AP: 0.7075
Epoch: 026, AUC: 0.6432, AP: 0.7097
Epoch: 027, AUC: 0.6508, AP: 0.7131
Epoch: 028, AUC: 0.6630, AP:

# Graph Variational AutoEncoder (GVAE)

In [31]:
from torch_geometric.nn import VGAE

In [32]:
dataset = Planetoid("\..", "CiteSeer", transform=T.NormalizeFeatures())
data = dataset[0]
data.train_mask = data.val_mask = data.test_mask = data.y = None
data = train_test_split_edges(data)


class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VariationalGCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv_mu = GCNConv(2 * out_channels, out_channels, cached=True)
        self.conv_logstd = GCNConv(2 * out_channels, out_channels, cached=True)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

C:\Users\Travail\anaconda3\envs\pyg\lib\site-packages\torch_geometric\deprecation.py:13: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [33]:
out_channels = 2
num_features = dataset.num_features
epochs = 300


model = VGAE(VariationalGCNEncoder(num_features, out_channels))  # new line

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [34]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    
    loss = loss + (1 / data.num_nodes) * model.kl_loss()  # new line
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [35]:
writer = SummaryWriter('runs/VGAE_experiment_'+'2d_300_epochs')

for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    
    
    writer.add_scalar('auc train',auc,epoch) # new line
    writer.add_scalar('ap train',ap,epoch)   # new line

Epoch: 001, AUC: 0.5810, AP: 0.5937
Epoch: 002, AUC: 0.5848, AP: 0.6051
Epoch: 003, AUC: 0.5925, AP: 0.6193
Epoch: 004, AUC: 0.5911, AP: 0.6188
Epoch: 005, AUC: 0.5947, AP: 0.6212
Epoch: 006, AUC: 0.5976, AP: 0.6243
Epoch: 007, AUC: 0.5999, AP: 0.6267
Epoch: 008, AUC: 0.6009, AP: 0.6278
Epoch: 009, AUC: 0.6027, AP: 0.6291
Epoch: 010, AUC: 0.6041, AP: 0.6302
Epoch: 011, AUC: 0.6044, AP: 0.6304
Epoch: 012, AUC: 0.6050, AP: 0.6311
Epoch: 013, AUC: 0.6049, AP: 0.6313
Epoch: 014, AUC: 0.6042, AP: 0.6309
Epoch: 015, AUC: 0.6038, AP: 0.6312
Epoch: 016, AUC: 0.6032, AP: 0.6314
Epoch: 017, AUC: 0.6025, AP: 0.6312
Epoch: 018, AUC: 0.6016, AP: 0.6310
Epoch: 019, AUC: 0.5994, AP: 0.6294
Epoch: 020, AUC: 0.5959, AP: 0.6282
Epoch: 021, AUC: 0.5960, AP: 0.6286
Epoch: 022, AUC: 0.5992, AP: 0.6306
Epoch: 023, AUC: 0.6041, AP: 0.6341
Epoch: 024, AUC: 0.6081, AP: 0.6371
Epoch: 025, AUC: 0.6110, AP: 0.6391
Epoch: 026, AUC: 0.6142, AP: 0.6409
Epoch: 027, AUC: 0.6168, AP: 0.6424
Epoch: 028, AUC: 0.6192, AP: